In [1]:
# Import libraries usefull

from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup

from collections import Counter
from operator import itemgetter
import pandas as pd

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [2]:
### Ajouter le dossier parent dans le chemin de recherche des modules 
# Solution retenue depuis cette doc:
# * https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder/11158224
## Autres solutions également proposées

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:
# It's made-home fonctions created by Francesco Beretta
# so they must to be in the same folder as this file.
### If you execute the script in the cell above, 
# the file can also be in the parent directory
import sparql_functions as spqf 

# Query economists and jurists

First step, we need datas about economists and jurists with theirs proprieties so we realise a SPARQL query. We need proprieties to realise the merge between three datasets:
  * Name
  
  * Birth date
  
  * Date of death
  
  * Place of Birth
  
  * Place of Death

In [5]:
### It's define the database ligne to use
pk_query = 1

# Connexion to the database
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

c = conn.cursor()

### It runs the query on the SQLite database to get the row values 
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# close connexion
conn.close()


In [6]:
# print(rc[2] +  "\n-----\n" + rc[4] +  "\n-----\n" +   rc[7]+  "\n\n\n------------------\n" +  rc[5] + "\n\n\n------------------\n")

In [7]:
### Execute the SPARQL query wrapped in the function in the library _sparql_functions.py_
# The first setting correspond to SPARQL Endpoint, the seconde to the query
q = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [33]:
### This fonction retrieves and shapes time

# definition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# execution
timestamp_formatted_for_file_name()

'20210427_151010'

In [34]:
# Define the file addresses, the existing one and new one
original_db = 'data/sparql_queries.db'

timestamped_db_copy = 'data/sparql_queries_' + timestamp_formatted_for_file_name() + '.sqlite'

In [35]:
## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/sparql_queries_20210427_151011.sqlite'

In [36]:
### store the answer of the SPARQL endpoint in the 'result'

conn = sql.connect(original_db)
c = conn.cursor()
values = (pk_query, str(r_bnf),timestamp_formatted_for_file_name())

# https://www.techonthenet.com/sqlite/functions/now.php
c.execute("INSERT INTO result (fk_query, result, timestmp) VALUES (?,?,?)", values)
# commit the insertion and close the database
# !! REQUIRE to be enabled to commit to the SQLite database !!
#conn.commit()
conn.close()

In [3]:
### Inspect the resultat after an insert

# Choose the row of the database to get
pk_result = ('3')

# connexion to the database
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

### execute the query on the SQLite database to retrieve the values of the row
c = conn.cursor()
c.execute('SELECT * FROM result WHERE pk_result = ?', pk_result)
result_q = c.fetchone()

# close connexion
conn.close()
# result_q[3]

In [4]:
### Transform string to dict
## Doc.:
# https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary
import ast
d = ast.literal_eval(result_q[3])
type(d)

dict

In [16]:
##### Transform the result into a list with a fonction of the library #####

#### Result of the BnF query
#r_bnf = [l for l in spqf.sparql_result_to_list(d)]
#print(len(r_bnf))
#r_bnf[:10]
#### Result of the Dbpedia query
#r_dbp = [l for l in spqf.sparql_result_to_list(d)]
#print(len(r_dbp))
#r_dbp[:10]
#### Result of the Dbpedia query (ressource "Lawyer")
#r_dbp_l = [l for l in spqf.sparql_result_to_list(d)]
#print(len(r_dbp_l))
#r_dbp_l[:10]
#### Result of the Wikidata query
#r_wk = [l for l in spqf.sparql_result_to_list(d)]
#print(len(r_wk))
#r_wk[:10]

-------------------------------

# Dataframes

The script below serves to change lists into a dataframe.

### BnF

In [ ]:
df_bnf = pd.DataFrame(r_bnf, columns=['uri_bnf','name_bnf','viaf_bnf','birthDate_bnf','deathDate_bnf' , 'placeOfBirth_bnf','placeOfDeath_bnf','bio_bnf'])
print(len(df_bnf))
df_bnf.fillna('')

df_bnf[:10]

In [4]:
endpoint = "http://192.168.0.21:7200/repositories/economistes/statements"

In [5]:
insert = """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  ome:  <https://ontome.net/class/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>


INSERT {GRAPH <http://economists.org/import_dbpedia> {?uri a ome:21.
        ?uri owl:sameAs ?person. } 
}
WHERE
  { SERVICE <https://dbpedia.org/sparql>
    { SELECT DISTINCT ?person
       WHERE
    {
   
    
    { ?person  a                     dbo:Economist }
    UNION
      { ?person  dbp:profession  dbr:Economist }
    UNION
      { ?person  dbp:occupation  dbr:Economist }
    UNION
      { ?person  dbp:field  dbr:Economist }
    UNION
      { ?person  dbp:fields  dbr:Economist }
    UNION
      { ?person  dbp:discipline  dbr:Economist }
    UNION
      { ?person  dbo:profession  dbr:Economist }
    UNION
      { ?person  dbo:occupation  dbr:Economist }
    UNION
      { ?person  dbo:academicDiscipline  dbr:Economist }
  }

}
            BIND(uuid() AS ?uri)
}

"""

In [ ]:
PREFIX  ome:  <https://ontome.net/class/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


INSERT DATA {GRAPH <http://economists.org/import_dbpedia> 
    {ome:21 rdfs:label 'Person';
    a rdfs:class.} 
}

In [ ]:
PREFIX  ome:  <https://ontome.net/class/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    
INSERT DATA {GRAPH <http://economists.org/import_dbpedia> 
    {ome:363 rdfs:label 'Geographical Place';
    a rdfs:class.} 
}

In [26]:
insert = """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  ome:  <https://ontome.net/class/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>


INSERT {GRAPH <http://economists.org/import_dbpedia> 
#  CONSTRUCT  
    {?uri rdfs:label ?label.
         } 
}
WHERE
{ ?uri owl:sameAs ?person.

  { SERVICE <https://dbpedia.org/sparql>
    { SELECT DISTINCT ?person ?label
       WHERE
    {
     
         ?person rdfs:label ?label
     {
    { ?person  a                     dbo:Economist }
    UNION
      { ?person  dbp:profession  dbr:Economist }
    UNION
      { ?person  dbp:occupation  dbr:Economist }
    UNION
      { ?person  dbp:field  dbr:Economist }
    UNION
      { ?person  dbp:fields  dbr:Economist }
    UNION
      { ?person  dbp:discipline  dbr:Economist }
    UNION
      { ?person  dbo:profession  dbr:Economist }
    UNION
      { ?person  dbo:occupation  dbr:Economist }
    UNION
      { ?person  dbo:academicDiscipline  dbr:Economist }
  }
  FILTER (LANG(?label) in ('en', 'fr'))             
}

}
        }

}



"""

In [37]:
### birthDate

insert = """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  ome:  <https://ontome.net/class/>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>

INSERT {
  GRAPH <http://economists.org/import_dbpedia> {
    ?uri dbo:birthDate ?birthDate .
  }
}
WHERE
  { ?uri  owl:sameAs  ?person
    { SERVICE <https://dbpedia.org/sparql>
        { SELECT DISTINCT  ?person ?birthDate
          WHERE
            { ?person  dbo:birthDate  ?birthDate
              {   { ?person  a                     dbo:Economist }
                UNION
                  { ?person  dbp:profession  dbr:Economist }
                UNION
                  { ?person  dbp:occupation  dbr:Economist }
                UNION
                  { ?person  dbp:field  dbr:Economist }
                UNION
                  { ?person  dbp:fields  dbr:Economist }
                UNION
                  { ?person  dbp:discipline  dbr:Economist }
                UNION
                  { ?person  dbo:profession  dbr:Economist }
                UNION
                  { ?person  dbo:occupation  dbr:Economist }
                UNION
                  { ?person  dbo:academicDiscipline  dbr:Economist }
              }
            }
        }
    }
  }
"""

In [42]:
### birthPlace

insert = """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  ome:  <https://ontome.net/class/>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>

INSERT {
  GRAPH <http://economists.org/import_dbpedia> {
    ?uri dbo:birthPlace ?birthPlace .
    ?birthPlace a ome:363.
  }
}
WHERE
  { ?uri  owl:sameAs  ?person
    { SERVICE <https://dbpedia.org/sparql>
        { SELECT DISTINCT  ?person ?birthPlace
          WHERE
            { ?person  dbo:birthPlace  ?birthPlace
              {   { ?person  a                     dbo:Economist }
                UNION
                  { ?person  dbp:profession  dbr:Economist }
                UNION
                  { ?person  dbp:occupation  dbr:Economist }
                UNION
                  { ?person  dbp:field  dbr:Economist }
                UNION
                  { ?person  dbp:fields  dbr:Economist }
                UNION
                  { ?person  dbp:discipline  dbr:Economist }
                UNION
                  { ?person  dbo:profession  dbr:Economist }
                UNION
                  { ?person  dbo:occupation  dbr:Economist }
                UNION
                  { ?person  dbo:academicDiscipline  dbr:Economist }
              }
            }
        }
    }
  }
"""

In [ ]:
SELECT  ?p (count(*) AS ?eff)
WHERE
  { { GRAPH <http://economists.org/import_dbpedia>
        { ?s  ?p  ?t }
    }
  }
GROUP BY ?p

In [ ]:
### Metadata using graph

PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  omc:  <https://ontome.net/class/>
PREFIX  omp:  <https://ontome.net/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dcmi: <http://purl.org/dc/terms/>
PREFIX moc: <http://myontology.org/>


INSERT {  
    GRAPH ?g_uri {
       ?uri   omp:1599 ?birthPlace.                   
    }
    ?uri a omc:61;
         omp:86 ?person.
     ?g_uri dcmi:source <http://dbpedia.org/>;
          moc:certainty 2.  
    
}


WHERE
{ SELECT ?person ?birthPlace ?uri ?g_uri
    {SERVICE <repository:economistes> {
        ?person dbo:birthPlace ?birthPlace.
    }
      BIND(uuid() AS ?uri)
    BIND(uuid() AS ?g_uri)
}
    LIMIT 10
}
    


In [45]:
### metadata using RDF STAR

PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  omc:  <https://ontome.net/class/>
PREFIX  omp:  <https://ontome.net/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dcmi: <http://purl.org/dc/terms/>
PREFIX moc: <http://myontology.org/>


INSERT { 
    ### Pro insert one graph -> metadata + rollback !
    GRAPH moc:g3 {
    ?person a omc:21.
    ?birth a omc:61.
    ?birth omp:86 ?person.
     <<?birth   omp:1599 ?birthPlace >> dcmi:source <http://dbpedia.org/>;
          moc:certainty 2.  
    }
    }

WHERE
{ SELECT ?person ?birthPlace ?birth
    {SERVICE <repository:economistes> {
        ?person dbo:birthPlace ?birthPlace.
    }
    BIND(uuid() AS ?birth)

}
    LIMIT 10
}
    
----


Query : 
    
    
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  omc:  <https://ontome.net/class/>
PREFIX  omp:  <https://ontome.net/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dcmi: <http://purl.org/dc/terms/>
PREFIX moc: <http://myontology.org/>


SELECT ?person ?p ?birth ?o1
WHERE { 
    GRAPH moc:g3 {
        ?person a omc:21.
?birth ?p ?person.
        <<?birth ?p1 ?o1>>  moc:certainty 2. 
        
    }
    }
    


SyntaxError: invalid syntax (<ipython-input-45-e5041a7c045b>, line 3)

In [33]:
endpoint = "http://192.168.0.21:7200/repositories/economistes/statements"

In [43]:
### Execute the SPARQL query wrapped in the function in the library _sparql_functions.py_
# The first setting correspond to SPARQL Endpoint, the seconde to the query
try:
    sparql = SPARQLWrapper(endpoint)
    sparql.setCredentials("admin", "root")
    sparql.setQuery(insert)
    sparql.setMethod('POST')
    # STOP #    
    rc = sparql.query()
except Exception as e:
    print(e)

In [23]:
dir(rc.response.readlines())

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [25]:
for l in rc.response.readlines():
    print (l)